# Write a Table with synthetic bounding boxes

<div style="display: inline-flex; align-items: center; gap: 10px;">
        <a href="https://colab.research.google.com/github/3lc-ai/notebook-examples/blob/main/write-bb-table.ipynb"
        target="_blank"
            style="background-color: transparent; text-decoration: none; display: inline-flex; align-items: center;
            padding: 5px 10px; font-family: Arial, sans-serif;"> <img
            src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab" style="height: 30px;
            vertical-align: middle;box-shadow: none;"/>
        </a> <a href="https://github.com/3lc-ai/notebook-examples/blob/main/write-bb-table.ipynb"
            style="text-decoration: none; display: inline-flex; align-items: center; background-color: #ffffff; border:
            1px solid #d1d5da; border-radius: 8px; padding: 2px 10px; color: #333; font-family: Arial, sans-serif;">
            <svg aria-hidden="true" focusable="false" role="img" class="octicon octicon-mark-github" viewBox="0 0 16 16"
            width="20" height="20" fill="#333"
            style="display:inline-block;user-select:none;vertical-align:text-bottom;overflow:visible; margin-right:
            8px;">
                <path d="M8 0c4.42 0 8 3.58 8 8a8.013 8.013 0 0 1-5.45 7.59c-.4.08-.55-.17-.55-.38 0-.27.01-1.13.01-2.2
                0-.75-.25-1.23-.54-1.48 1.78-.2 3.65-.88 3.65-3.95 0-.88-.31-1.59-.82-2.15.08-.2.36-1.02-.08-2.12 0
                0-.67-.22-2.2.82-.64-.18-1.32-.27-2-.27-.68 0-1.36.09-2 .27-1.53-1.03-2.2-.82-2.2-.82-.44 1.1-.16
                1.92-.08 2.12-.51.56-.82 1.28-.82 2.15 0 3.06 1.86 3.75 3.64 3.95-.23.2-.44.55-.51
                1.07-.46.21-1.61.55-2.33-.66-.15-.24-.6-.83-1.23-.82-.67.01-.27.38.01.53.34.19.73.9.82 1.13.16.45.68
                1.31 2.69.94 0 .67.01 1.3.01 1.49 0 .21-.15.45-.55.38A7.995 7.995 0 0 1 0 8c0-4.42 3.58-8 8-8Z"></path>
            </svg> <span style="vertical-align: middle; color: #333;">Open in GitHub</span>
        </a>
</div>

This notebook demonstrates how to write a `tlc.Table` from scratch using a `tlc.TableWriter`.

Using randomly generated bounding boxes as an example, we show how to format the data and construct the schema in a way
that makes the resulting table viewable in the 3LC Dashboard.

In [ ]:
PROJECT_NAME = "Table Writer Examples"
DATASET_NAME = "Synthetic Bounding Boxes"
TABLE_NAME = "bb-table"
TLC_PUBLIC_EXAMPLES_DEVELOPER_MODE = True
INSTALL_DEPENDENCIES = False
MAX_LABEL = 10  # Define the maximum label value

In [ ]:
%%capture
if INSTALL_DEPENDENCIES:
    %pip --quiet install 3lc

In [ ]:
### HIDDEN CELL ###

# Reloads all modules every time before executing the Python code.
%load_ext autoreload
%autoreload 2

# Ensure notebook_tests on PATH
import os
import sys

sys.path.append('..')
import notebook_tests

# Optionally override the default test data path
if path := os.getenv("TLC_PUBLIC_EXAMPLES_TEST_DATA_PATH"):
    print(f"Using test data path: {path}")
    TEST_DATA_PATH = path

# Prints the current 3lc configuration
!3lc config --list

## Setup the TableWriter

First, we need to import the `tlc` library and create a `tlc.TableWriter` object.
We will provide a `tlc.Schema` to the table writer in a later cell.

In [ ]:
import tlc

bb_schema = tlc.BoundingBoxListSchema(
    label_value_map={float(i): tlc.MapElement(f"label_{i}") for i in range(MAX_LABEL + 1)},
    x0_number_role=tlc.NUMBER_ROLE_BB_CENTER_X,
    y0_number_role=tlc.NUMBER_ROLE_BB_CENTER_Y,
    x1_number_role=tlc.NUMBER_ROLE_BB_SIZE_X,
    y1_number_role=tlc.NUMBER_ROLE_BB_SIZE_Y,
    x0_unit="relative",
    y0_unit="relative",
    x1_unit="relative",
    y1_unit="relative",
    include_segmentation=False,
)

schemas = {
    "image": tlc.Schema(value=tlc.ImageUrlStringValue()),
    "bounding_boxes": bb_schema
}

table_writer = tlc.TableWriter(
    project_name=PROJECT_NAME,
    dataset_name=DATASET_NAME,
    table_name=TABLE_NAME,
    column_schemas=schemas,
    if_exists="overwrite",
)

In [ ]:
# Helper function for creating bounding boxes
import random


def create_dummy_bounding_box():
    # Generate random center (x0, y0) and size (x1, y1) for the bounding box
    # This is yolo format: [label, x0, y0, x1, y1]
    x0, y0 = random.uniform(0.1, 0.9), random.uniform(0.1, 0.9)
    x1, y1 = random.uniform(0.05, 0.2), random.uniform(0.05, 0.2)

    # Generate a random label
    label = random.randint(0, MAX_LABEL)

    return {"x0": x0, "y0": y0, "x1": x1, "y1": y1, "label": label}

def create_dummy_bounding_boxes():
    # Generate a random number of bounding boxes between 3 and 5
    num_boxes = random.randint(3, 5)
    bounding_boxes = [create_dummy_bounding_box() for _ in range(num_boxes)]

    # Update image dimensions
    image_height = random.randint(100, 500)
    image_width = random.randint(100, 500)

    return {"image_height": image_height, "image_width": image_width, "bb_list": bounding_boxes}


## Create Table Data

In [ ]:
# Our table will contain four rows. Each row will have two columns: a image column, and a bounding boxes column.
table_rows = {
    "image": [],
    "bounding_boxes": []
}

for i in range(4):
    table_rows["image"].append(f"image_{i}.jpg") # A nonexistent image file, for demonstration purposes
    table_rows["bounding_boxes"].append(create_dummy_bounding_boxes())

## Construct a Schema for the Table

A column of bounding boxes in 3LC is represented as a dictionary of the form:

```python
{
    "image_width": float,
    "image_height": float,
    "bb_list": [
        {
            "x0": float,  # First "horizontal" coordinate
            "x1": float,  # Second "horizontal" coordinate
            "y0": float,  # First "vertical" coordinate
            "y1": float,  # Second "vertical" coordinate
            "label": str  # Label of the bounding box
        },
        ...
    ]
}
```

## Write the table

## Write the table

In [ ]:
table_writer.add_batch(table_rows)
table = table_writer.finalize()

In [ ]:
# Inspect the first row
table[0]